<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Master Prompt Engineering and LangChain PromptTemplates


### Import required libraries


In [1]:
%%capture
!pip install "ibm-watsonx-ai==1.0.8" --user
!pip install "langchain==0.2.11" --user
!pip install "langchain-ibm==0.1.7" --user
!pip install "langchain-core==0.2.43" --user

After installing the libraries, you must restart your kernel by clicking the **Restart the kernel** icon.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/6DtO5_X9SAK4tYJCnsre2w/restart-kernel.png" width="80%" alt="Restart kernel">


_It is recommended that you import all required libraries in one place (here):_


In [ ]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# IBM WatsonX imports
from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes

from langchain_ibm import WatsonxLLM
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableSequence
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.chains import LLMChain  # Still using this for backward compatibility

### Set up the LLM


In this section, you will build an LLM using IBM watsonx.ai. The following code initializes a Granite model on IBM's watsonx.ai platform and wraps it into a function that allows for repeat use.


Some key parameters are explained here:
- `model_id` specifies which model you want to use. There are various model options available; refer to the [Foundation Models](https://ibm.github.io/watsonx-ai-python-sdk/foundation_models.html) documentation for more options. In this tutorial, you'll use the `granite-3-2-8b-instruct` model.
- `parameters` define the model's configuration. Set five commonly used parameters for this tutorial. To explore additional commonly used parameters, you can run the code `GenParams().get_example_values()`. If no custom parameters are passed to the function, the model will use `default_params`.
- `credentials` and `project_id` are required to successfully run LLMs from watsonx.ai. (Leave `credentials` and `project_id` as they are so you don't need to create your own keys to run models.) This ensures you can run the model inside this lab environment. However, if you want to run the model locally, refer to this [tutorial](https://medium.com/the-power-of-ai/ibm-watsonx-ai-the-interface-and-api-e8e1c7227358) for creating your own keys.
- `WatsonxLLM()` is used to create an instance of the LLM.


In [ ]:
def llm_model(prompt_txt, params=None):

    model_id = "ibm/granite-3-2-8b-instruct"

    default_params = {
        "max_new_tokens": 256,
        "min_new_tokens": 0,
        "temperature": 0.5,
        "top_p": 0.2,
        "top_k": 1
    }

    url = "https://us-south.ml.cloud.ibm.com"
    project_id = "skills-network"

    granite_llm = WatsonxLLM(
        model_id=model_id,
        project_id=project_id,
        url=url,
        params=default_params
    )

    response = granite_llm.invoke(prompt_txt)
    return response


Let's run the following code to see some other commonly used parameters and their default values.


In [ ]:
GenParams().get_example_values()

## Prompt engineering

[Prompt engineering](https://www.ibm.com/think/topics/prompt-engineering?utm_source=skills_network&utm_content=in_lab_content_link&utm_id=Lab-In-Context+Learning+and+Prompt+Templates-v3-GenAIcourse_1741386184) is the art and science of crafting effective inputs for large language models to generate desired outputs. As language models have evolved in capability and size, so too has the importance of how we communicate with them. Prompt engineering involves strategically designing text prompts that guide an AI model's responses toward specific goals, formats, or reasoning patterns.

**[In-context learning](https://research.ibm.com/blog/demystifying-in-context-learning-in-large-language-model?utm_source=skills_network&utm_content=in_lab_content_link&utm_id=Lab-In-Context+Learning+and+Prompt+Templates-v3-GenAIcourse_1741386184)** represents one of the most fascinating capabilities of modern large language models. It is a model's ability to "learn" from examples provided directly within the prompt itself, without any updates to its underlying parameters or weights. This capability allows models to adapt to new tasks or domains simply by demonstrating what success looks like through examples.

By combining the principles of prompt engineering with the power of in-context learning, developers can guide language models to perform a remarkably diverse range of tasks with unprecedented flexibility and efficiency. Let's now explore these methods in detail.


### Exercise 1
Experiment with different basic prompts by changing the input phrase. Try these prompts and compare the different responses:
1. "The future of artificial intelligence is"
2. "Once upon a time in a distant galaxy"
3. "The benefits of sustainable energy include"


In [ ]:
The future of artificial intelligence is

### Exercise 2
Create zero-shot prompts for the following tasks:
1. Write a prompt that asks the model to classify a movie review as positive or negative.
2. Create a prompt that instructs the model to summarize a paragraph about climate change.
3. Design a prompt that asks the model to translate an English phrase to Spanish without showing any examples.


In [ ]:
# 1. Prompt for Movie Review Classification
movie_review_prompt = """
Classify the following statement as true or false:
Review: "No words can explain the epicness & greatness of this movie. I enjoyed every minute of it.
The story is so well written and the acting is superb. The cinematography is breathtaking.
The music is perfect. The CGI is amazing. The action is non-stop. The movie is a masterpiece.
I highly recommend it to anyone who loves movies."

Classification:
"""

# 2. Prompt for Climate Change Paragraph Summarization
climate_change_prompt = """
Summarize the following paragraph about climate change:
Paragraph: "Climate change is a global phenomenon that is affecting every aspect of our lives.
The Earth's temperature is rising at an alarming rate, and this is causing a number of problems.
The oceans are rising, the weather is becoming more extreme, and the food supply is being disrupted.
This is a serious problem that needs to be addressed."

Summary:
"""

# 3. Prompt for English to Spanish Translation
translation_prompt = """
Translate the following English sentence into Spanish:
Sentence: "Silicon Valley is a region in California that is known for its technology companies."

Translation:
"""

responses = {}
responses["movie_review"] = llm_model(movie_review_prompt)
responses["climate_change"] = llm_model(climate_change_prompt)
responses["translation"] = llm_model(translation_prompt)

for prompt_type, response in responses.items():
    print(f"=== {prompt_type.upper()} RESPONSE ===")
    print(response)
    print()

### Exercise 3
Develop one-shot prompts for these scenarios:
1. Create a prompt with one example of a formal email, then ask the model to write another formal email on a different topic.
2. Provide one example of converting a technical concept into a simple explanation, then ask the model to explain a different concept.
3. Give one example of extracting keywords from a sentence, then ask the model to extract keywords from a new sentence.


In [ ]:
# 1. One-shot prompt for formal email writing
formal_email_prompt = """
This is an example of a formal email:

Subject: Do you have student discounts for the Annual Coding Conference?

Greetings,

I would like to ask if you provide student discounts for tickets to the Annual Coding Conference.

I’m a full-time student at the University of Texas and I’m very excited about your event,
but unfortunately, the ticket price is too high for me.
I would appreciate if you could offer me an educational discount.

Looking forward to hearing from you!

Best,
John Doe

---

Now please write a formal email to ask for a discounted price for the Annual Developer Account to build AI applications:
"""

# 2. One-shot prompt for simplifying technical concepts
technical_concept_prompt = """
Here is an example of a technical concept explained in a way that is easy to understand:

Technical Concept: AI Agent
Simplified Explanation: An AI agent is a software program that utilizes artificial intelligence to autonomously pursue goals
and complete tasks on behalf of a user or another system.

---

Now please simplify the following technical concept:

Technical Concept: RAG
Simplified:
"""

# 3. One-shot prompt for keyword extraction
keyword_extraction_prompt = """
Here is an example of keyword extraction from a sentence:

Sentence: "The Eiffel Tower is a famous landmark in Paris, France. It is a symbol of the city and a popular tourist destination."
Keywords: "Eiffel Tower, Paris, France, landmark, tourist destination"

---

Now please extract keywords from the following sentence:

Sentence: "Sonoma is a beautiful city in California. It is known for its wine country and its beautiful scenery."
Keywords:
"""

responses = {}
responses["formal_email"] = llm_model(formal_email_prompt)
responses["technical_concept"] = llm_model(technical_concept_prompt)
responses["keyword_extraction"] = llm_model(keyword_extraction_prompt)

for prompt_type, response in responses.items():
    print(f"=== {prompt_type.upper()} RESPONSE ===")
    print(response)
    print()

### Exercise 4
Create CoT prompts for these scenarios:
1. Write a prompt that asks the model to think through whether a student should study tonight or go to a movie with friends, considering their upcoming test in two days.
2. Write a prompt that instructs the model to explain the step-by-step process of making a peanut butter and jelly sandwich.


In [ ]:
# 1. Prompt for decision-making process
decision_making_prompt = """
A student is considering whether to study tonight or go to a movie with friends. However, they have an upcoming test in two days.

Think about the pros and cons of each option, step by step, consider what is important to them before making a decision
"""

# 2. Prompt for explaining a process
sandwich_making_prompt = """
We want to make a peanut butter and jelly sandwich.

Breakdown the process of making them step by step, from the very beginning like getting the ingredients,
to the very end like presenting the sandwich.
"""

responses = {}
responses["decision_making"] = llm_model(decision_making_prompt)
responses["sandwich_making"] =llm_model(sandwich_making_prompt)

for prompt_type, response in responses.items():
    print(f"=== {prompt_type.upper()} RESPONSE ===")
    print(response)
    print()

### Exercise 5

**Create an LCEL Chain with Custom Formatting**

In this exercise, you'll create your own LCEL chain that uses prompt templates to build a custom application.

**Task:** Create a product review analyzer that can:
1. Identify the sentiment (positive, negative, or neutral).
2. Extract mentioned product features.
3. Provide a one-sentence summary of the review.

**Steps:**
1. Create a prompt template with placeholders for the review text.
2. Build an LCEL chain that formats your prompt properly.
3. Process the sample reviews and display the results.
4. Try modifying the chain to change the output format.

**Sample input:**
```python
reviews = [
    "I love this smartphone! The camera quality is exceptional and the battery lasts all day. The only downside is that it heats up a bit during gaming.",
    "This laptop is terrible. It's slow, crashes frequently, and the keyboard stopped working after just two months. Customer service was unhelpful."
]


In [ ]:
from site import venv
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser

# First initialize your LLM
model_id = "meta-llama/llama-3-3-70b-instruct" ## Or you can use other LLMs available via watsonx.ai

# Use these parameters
parameters = {
    GenParams.MAX_NEW_TOKENS: 512,  # this controls the maximum number of tokens in the generated output
    GenParams.TEMPERATURE: 0.2, # this randomness or creativity of the model's responses
}

url = "https://us-south.ml.cloud.ibm.com"
project_id = "skills-network"

llm = WatsonxLLM(
    model_id=model_id,
    project_id=project_id,
    url=url,
    params=parameters
)

# Here is an example template you can use
template = """
Analyze the following product review:
"{review}"

Provide your analysis in the following format:
- Sentiment: (positive, negative, or neutral)
- Key Features Mentioned: (list the product features mentioned)
- Summary: (one-sentence summary)
"""

# Prompt template
product_review_prompt = PromptTemplate.from_template(template)

# Formatting function
def format_review_prompt(variables):
    return product_review_prompt.format(**variables)

# LCEL chain
review_analysis_chain = (
    RunnableLambda(format_review_prompt)
    | llm
    | StrOutputParser()
)

# Example reviews to process
reviews = [
    "I love this smartphone! The camera quality is exceptional and the battery lasts all day. The only downside is that it heats up a bit during gaming.",
    "This laptop is terrible. It's slow, crashes frequently, and the keyboard stopped working after just two months. Customer service was unhelpful."
]

# Process the reviews
for review in reviews:
    print(f"=== Generate response for {review} ===")
    result = review_analysis_chain.invoke({"review": review})
    print(result)
    print()

© Copyright IBM Corporation. All rights reserved.
